## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="gemma2-9b-It", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x767581ad9be0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x767581ada7b0>, model_name='gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is krish and im a chief ai engineer")])

AIMessage(content="Hello Krish! It's a pleasure to meet you.  That's a fascinating role!  \n\nAs a large language model, I'm always eager to learn more about how AI is being used in the world. What kind of projects are you working on?  \n\nPerhaps you could tell me about:\n\n* **The most exciting challenge you're facing in your work.**\n* **A recent breakthrough or accomplishment you're proud of.**\n* **What you think the future holds for AI engineering.**\n\n\nI'm excited to hear your insights!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 22, 'total_tokens': 141, 'completion_time': 0.216363636, 'prompt_time': 0.001325851, 'queue_time': 0.248999548, 'total_time': 0.217689487}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2329ca5a-a42d-476d-ad0c-e1143a1cc27e-0', usage_metadata={'input_tokens': 22, 'output_tokens': 119, 'tota

In [ ]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content = "Hi my name is keshav, and im a chief ai engineer"),
    AIMessage(content="Hello Keshav! It's a pleasure to meet you.  That's a fascinating role!  \n\nAs a large language model, I'm always eager to learn more about how AI is being used in the world. What kind of projects are you working on?  \n\nPerhaps you could tell me about:\n\n* **The most exciting challenge you're facing in your work.**\n* **A recent breakthrough or accomplishment you're proud of.**\n* **What you think the future holds for AI engineering.**\n\n\nI'm excited to hear your insights!\n"),
    HumanMessage(content="hey, whats my name, what do i do?")
])

AIMessage(content="You are Keshav, and you are a Chief AI Engineer!  \n\nIs there anything else you'd like to know or discuss?  😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 161, 'total_tokens': 195, 'completion_time': 0.061818182, 'prompt_time': 0.004546752, 'queue_time': 0.247997367, 'total_time': 0.066364934}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d0979814-2956-4ad6-b918-b155ff6b9a33-0', usage_metadata={'input_tokens': 161, 'output_tokens': 34, 'total_tokens': 195})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: int)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config = {"configurable": {"session_id": "chat1"}}

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is keshav and im an ai engineer")],
    config=config
)

response.content

"Hi Keshav! \n\nIt's great to meet you. Being an AI engineer is such a cutting-edge field. What kind of projects are you working on?  \n\nI'm always interested to hear what people are doing in the world of AI.  😄 \n\n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="whats my name")],
    config=config
)

AIMessage(content='Your name is Keshav.  😊  I remember! \n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 171, 'total_tokens': 188, 'completion_time': 0.030909091, 'prompt_time': 0.004450179, 'queue_time': 0.25329589, 'total_time': 0.03535927}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4cb92544-b1c9-4373-8376-5b1d059067ce-0', usage_metadata={'input_tokens': 171, 'output_tokens': 17, 'total_tokens': 188})

In [13]:
# change the session id (config)
config1= {"configurable": {"session_id": "chat2"}}
with_message_history.invoke(
    [HumanMessage(content="whats my name")],
    config=config1
)

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name.\n\nIf you'd like to tell me your name, I'd be happy to use it! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 12, 'total_tokens': 55, 'completion_time': 0.078181818, 'prompt_time': 0.00116769, 'queue_time': 0.24656495, 'total_time': 0.079349508}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--bf5083e7-31f7-40f5-a8f1-070bcf084e2a-0', usage_metadata={'input_tokens': 12, 'output_tokens': 43, 'total_tokens': 55})